In [ ]:
!nvidia-smi

In [ ]:
# sync python module
%load_ext autoreload
%autoreload 2


Infer

In [ ]:
workspace_dir = '/CardiacSegV2'

import os
import sys
from pathlib import PurePath
sys.path.append(workspace_dir)

import pandas as pd

from datasets.chgh_dataset import get_data_dicts
from data_utils.utils import get_pid_by_file
from data_utils.visualization import show_img_lbl, show_img_lbl_pred, show_img_lbl_preds, show_img_lbl_preds_overlap

from expers.infer_utils import get_tune_model_dir, get_data_path, get_pred_path

model_name = 'unetr_pp'
data_name = 'chgh'
sub_data_dir_name = 'infer'
exp_name = 'AICUP_training'
data_dict_file_name = 'AICUP_training.json'

root_exp_dir = os.path.join(
    workspace_dir,
    'exps',
    'exps',
    model_name,
    data_name,
    'tune_results'
)

root_data_dir = os.path.join(
    workspace_dir,
    'dataset',
    data_name
)

data_dir = os.path.join(root_data_dir, sub_data_dir_name)
model_dir = get_tune_model_dir(root_exp_dir, exp_name)

best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')

infer_dir = os.path.join(
    workspace_dir,
    'exps',
    'exps',
    model_name,
    data_name,
    'infers',
    exp_name,
)

print('\nbest model:',best_checkpoint)
print('infer dir:',infer_dir)


pred_img = []

for root, dirs, files in os.walk("/CardiacSegV2/dataset/chgh/infer", topdown=False):
  for name in files:
    pred_img.append(os.path.join(root, name))
    print(os.path.join(root, name))

pred_img.sort()

In [ ]:
for img_pth in pred_img:
  !python {workspace_dir}/expers/infer.py \
    --model_name=unetr_pp \
    --data_name={data_name} \
    --data_dir={data_dir} \
    --model_dir={model_dir} \
    --infer_dir={infer_dir} \
    --checkpoint={best_checkpoint} \
    --img_pth={img_pth} \
    --out_channels=4 \
    --patch_size=2 \
    --feature_size=32 \
    --drop_rate=0.1 \
    --depths 3 3 9 3 \
    --kernel_size 7 \
    --exp_rate 4 \
    --norm_name='layer' \
    --a_min=-42 \
    --a_max=423 \
    --space_x=0.7 \
    --space_y=0.7 \
    --space_z=1.0 \
    --roi_x=96 \
    --roi_y=96 \
    --roi_z=96 \
    --infer_post_process


In [ ]:
import shutil
import os

os.chdir("/CardiacSegV2/exps/exps/swinunetr/chgh/infers/AICUP_training")

zip_path = "/CardiacSegV2/contest1124_5bd88_00000"
shutil.make_archive(zip_path, 'zip', root_dir='.', base_dir='.')
